In [21]:
import pandas as pd
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 's3://aws-public-blockchain/v1.0/eth/blocks/date=2015-10-27'


files = fs.ls(directory_path)

parquet_files = [file for file in files if file.endswith('.parquet')]

file_path = 's3://' + parquet_files[0]

print(parquet_files[:5])

with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')


for x in df:
    print(x)


['aws-public-blockchain/v1.0/eth/blocks/date=2015-10-27/part-00000-b4880e90-ba35-432a-aa7c-3fef105d99dc-c000.snappy.parquet']
difficulty
hash
miner
nonce
number
size
timestamp
total_difficulty
base_fee_per_gas
gas_limit
gas_used
extra_data
logs_bloom
parent_hash
state_root
receipts_root
transactions_root
sha3_uncles
transaction_count
date
last_modified


In [6]:
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 's3://aws-public-blockchain/v1.0/eth/blocks/date=2015-07-30/part-00000-32767f69-9150-49ac-9c03-45f34b103c34-c000.snappy.parquet'


files = fs.ls(directory_path)


parquet_files = [file for file in files]

print(parquet_files[:5])


['aws-public-blockchain/v1.0/eth/blocks/date=2015-07-30/part-00000-32767f69-9150-49ac-9c03-45f34b103c34-c000.snappy.parquet']


In [57]:
from datetime import datetime, timedelta
import pandas as pd
import s3fs
import pyrqlite.dbapi2 as dbapi2
import psutil

connection = dbapi2.connect(
    host='localhost',
    port=4001,
)

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

if __name__ == "__main__":
    start_date = datetime(2023, 4, 1)  # Start date is today
    end_date = start_date + timedelta(days=2)  # End date is 6 months from now

    with connection.cursor() as cursor:
        cursor.execute('CREATE TABLE IF NOT EXISTS blocks (hsh text)')

    ct = 0
    memory_percent = 0
    disk_io = psutil.disk_io_counters()
    network_io = psutil.net_io_counters()
    cpu_percent = 0
    stTime = datetime.now()
    for single_date in date_range(start_date, end_date):
        #print(single_date.strftime("%Y-%m-%d"))
        dtt = single_date.strftime("%Y-%m-%d")

        fs = s3fs.S3FileSystem(anon=True)

        directory_path = 's3://aws-public-blockchain/v1.0/eth/blocks/date='+dtt

        files = fs.ls(directory_path)

        parquet_files = [file for file in files if file.endswith('.parquet')]

        file_path = 's3://' + parquet_files[0]

        # print(parquet_files[:5])

        with fs.open(file_path, mode='rb') as f:
            df = pd.read_parquet(f, engine='pyarrow')


        for x in df['nonce']:
            ct+=1

            try:
                with connection.cursor() as cursor:
                    cursor.executemany('INSERT INTO blocks(hsh) VALUES(?)', seq_of_parameters=((x)))
            finally:
                connection.close()
            if(ct>1000):
                break
    print(psutil.virtual_memory().percent,psutil.disk_io_counters(),psutil.net_io_counters(),psutil.cpu_percent(interval=1),datetime.now()-stTime)
        

76.7 sdiskio(read_count=87830101, write_count=48867051, read_bytes=3174632250880, write_bytes=2697905180672, read_time=54187482, write_time=2670766) snetio(bytes_sent=3209435121, bytes_recv=10674705813, packets_sent=13271518, packets_recv=19200742, errin=0, errout=0, dropin=11453246114, dropout=0) 14.5 0:00:06.282365
